Querying data that will be used to train the neural network
---
1) querying all coins for which will build a correlated neural network model

2) keeping only the data of interest in an array:

    t-m:   p1 p2 p3 p4 ... pn
    t-m+1: p1 p2 p3 p4 ... pn
    t-m+2: p1 p2 p3 p4 ... pn 
    ...
    t:     p1 p2 p3 p4 ... pn

3) the vars array will have shape (n currencies, m points)

In [1]:
import numpy as np

from cryptocompare import get_histo

"""
hour - 2000 points max
minute - 2000 points max
"""

vars = []
mhd = 'hour'
candle_width = 1
m = 2000
e = 'CCCAGG'

# Selected cryptos with higher market caps (https://coinmarketcap.com/)
currencies = ['BTC', 'ETH', 'XRP', 'BCH', 'LTC', 'NEO', 'XLM']
currencies = ['BTC', 'ETH', 'XRP']
n = len(currencies)
for coin in currencies:
    q = get_histo(mhd, coin, m, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        if d0['close'] == 0:
            raise ValueError('Invalid input for crypto %s: %s' % (coin, str(d0)))
        else:
            p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)
print('vars.shape', vars.shape)

vars.shape (3, 2000)


### Particioning blocks of data that will be used to build the neural network
---
1) determining the number of points for each block. The "block" means the number of candles that will be used to compute the next one

2) building vector of inputs (xs) and outputs (ys)

3) classifying responses for each cryptocurrency using the following scheme (only 2 classes, I tried with more classes but the prediction gets much worse):

    0 -->     y < threshold
    1 -->     y >= threshold
    

In [2]:
block = 100
xs = []
ys = []
for i in range(vars.shape[1] - (block+1)):
    part = vars[:, i:i+(block+1)]
    xs.append(part[:, :-1])
    ys.append(part[:, -1])
xs = np.array(xs)
print('xs.shape', xs.shape)
def classify(y):
    threshold = 0.02
    if y < threshold:
        return 0
    else:
        return 1

classes = {}
for i, currency in enumerate(currencies):
    tmp = []
    for y in ys:
        tmp.append(classify(y[i]))
    classes[currency] = np.array(tmp)
    

xs.shape (1899, 3, 100)


Shuffling samples for a better stochastic response
---

In [3]:
import random
rand_indices = np.arange(xs.shape[0])
random.shuffle(rand_indices)
xs = xs[rand_indices]
xs = xs.reshape(xs.shape[0], -1)
print('xs.shape', xs.shape)
for k in classes:
    classes[k] = classes[k][rand_indices]

xs.shape (1899, 300)


Selecting training set and testing set
---

In [4]:
train_size = 1500

xs_train = xs[:train_size]
xs_test = xs[train_size:]
classes_train = {}
classes_test = {}
for k in classes:
    classes_train[k] = classes[k][:train_size]
    classes_test[k] = classes[k][train_size:]

Training a simple Keras model
---

Reference: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [5]:
from keras.models import Sequential
from keras.layers import Dense

models = {}
for k in currencies:
    Y = classes_train[k]
    # create model
    model = Sequential()
    model.add(Dense(14, kernel_initializer='glorot_normal', input_dim=xs_train.shape[1], activation='relu'))
    model.add(Dense(8, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='glorot_normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    out = model.fit(xs_train, Y, epochs=250, batch_size=50)
    models[k] = model

Using TensorFlow backend.


Epoch 1/250
1500/1500 [==============================] - 0s 259us/step - loss: 0.6811 - acc: 0.8767
Epoch 2/250
1500/1500 [==============================] - 0s 25us/step - loss: 0.6497 - acc: 0.9047
Epoch 3/250
1500/1500 [==============================] - 0s 27us/step - loss: 0.6057 - acc: 0.9047
Epoch 4/250
1500/1500 [==============================] - 0s 26us/step - loss: 0.5443 - acc: 0.9047
Epoch 5/250
1500/1500 [==============================] - 0s 25us/step - loss: 0.4694 - acc: 0.9047
Epoch 6/250
1500/1500 [==============================] - 0s 27us/step - loss: 0.3959 - acc: 0.9047
Epoch 7/250
1500/1500 [==============================] - 0s 26us/step - loss: 0.3398 - acc: 0.9047
Epoch 8/250
1500/1500 [==============================] - 0s 25us/step - loss: 0.3112 - acc: 0.9047
Epoch 9/250
1500/1500 [==============================] - 0s 27us/step - loss: 0.2982 - acc: 0.9047
Epoch 10/250
1500/1500 [==============================] - 0s 25us/step - loss: 0.2926 - acc: 0.9047
Epoch 11

1500/1500 [==============================] - 0s 20us/step - loss: 0.0379 - acc: 0.9940
Epoch 84/250
1500/1500 [==============================] - 0s 20us/step - loss: 0.0372 - acc: 0.9940
Epoch 85/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0365 - acc: 0.9940
Epoch 86/250
1500/1500 [==============================] - 0s 19us/step - loss: 0.0359 - acc: 0.9940
Epoch 87/250
1500/1500 [==============================] - 0s 12us/step - loss: 0.0353 - acc: 0.9940
Epoch 88/250
1500/1500 [==============================] - 0s 29us/step - loss: 0.0348 - acc: 0.9940
Epoch 89/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0344 - acc: 0.9940
Epoch 90/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0340 - acc: 0.9940
Epoch 91/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0335 - acc: 0.9940
Epoch 92/250
1500/1500 [==============================] - 0s 19us/step - loss: 0.0331 - acc: 0.9940
Epoch 93/250


1500/1500 [==============================] - 0s 21us/step - loss: 0.0060 - acc: 0.9987
Epoch 165/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0057 - acc: 0.9987
Epoch 166/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0055 - acc: 0.9987
Epoch 167/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0053 - acc: 0.9987
Epoch 168/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0051 - acc: 0.9987
Epoch 169/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0048 - acc: 0.9987
Epoch 170/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0045 - acc: 0.9987
Epoch 171/250
1500/1500 [==============================] - 0s 29us/step - loss: 0.0043 - acc: 0.9993
Epoch 172/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0041 - acc: 0.9993
Epoch 173/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0038 - acc: 0.9993
Epoc

1500/1500 [==============================] - 0s 23us/step - loss: 1.8630e-04 - acc: 1.0000
Epoch 244/250
1500/1500 [==============================] - 0s 26us/step - loss: 1.8203e-04 - acc: 1.0000
Epoch 245/250
1500/1500 [==============================] - 0s 26us/step - loss: 1.7707e-04 - acc: 1.0000
Epoch 246/250
1500/1500 [==============================] - 0s 25us/step - loss: 1.7203e-04 - acc: 1.0000
Epoch 247/250
1500/1500 [==============================] - 0s 24us/step - loss: 1.6784e-04 - acc: 1.0000
Epoch 248/250
1500/1500 [==============================] - 0s 25us/step - loss: 1.6321e-04 - acc: 1.0000
Epoch 249/250
1500/1500 [==============================] - 0s 26us/step - loss: 1.5985e-04 - acc: 1.0000
Epoch 250/250
1500/1500 [==============================] - 0s 24us/step - loss: 1.5582e-04 - acc: 1.0000
Epoch 1/250
1500/1500 [==============================] - 0s 254us/step - loss: 0.6844 - acc: 0.9000
Epoch 2/250
1500/1500 [==============================] - 0s 26us/step - lo

...

1500/1500 [==============================] - 0s 25us/step - loss: 0.0200 - acc: 0.9980
Epoch 156/250
1500/1500 [==============================] - 0s 24us/step - loss: 0.0199 - acc: 0.9980
Epoch 157/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0196 - acc: 0.9980
Epoch 158/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0195 - acc: 0.9980
Epoch 159/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0193 - acc: 0.9980
Epoch 160/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0191 - acc: 0.9980
Epoch 161/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0190 - acc: 0.9980
Epoch 162/250
1500/1500 [==============================] - 0s 24us/step - loss: 0.0188 - acc: 0.9980
Epoch 163/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0186 - acc: 0.9980
Epoch 164/250
1500/1500 [==============================] - 0s 24us/step - loss: 0.0184 - acc: 0.9980
Epoc

1500/1500 [==============================] - 0s 21us/step - loss: 0.0118 - acc: 0.9980
Epoch 237/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0117 - acc: 0.9980
Epoch 238/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0117 - acc: 0.9980
Epoch 239/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0117 - acc: 0.9980
Epoch 240/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0116 - acc: 0.9980
Epoch 241/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0116 - acc: 0.9980
Epoch 242/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0115 - acc: 0.9980
Epoch 243/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0115 - acc: 0.9980
Epoch 244/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0114 - acc: 0.9980
Epoch 245/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0114 - acc: 0.9980
Epoc

Epoch 68/250
1500/1500 [==============================] - 0s 25us/step - loss: 0.0581 - acc: 0.9920
Epoch 69/250
1500/1500 [==============================] - 0s 24us/step - loss: 0.0555 - acc: 0.9920
Epoch 70/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0543 - acc: 0.9913
Epoch 71/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0521 - acc: 0.9927
Epoch 72/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0509 - acc: 0.9920
Epoch 73/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0476 - acc: 0.9920
Epoch 74/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0465 - acc: 0.9920
Epoch 75/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0450 - acc: 0.9927
Epoch 76/250
1500/1500 [==============================] - 0s 24us/step - loss: 0.0435 - acc: 0.9927
Epoch 77/250
1500/1500 [==============================] - 0s 24us/step - loss: 0.0420 - acc: 0.9927


1500/1500 [==============================] - 0s 21us/step - loss: 0.0015 - acc: 1.0000
Epoch 150/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0014 - acc: 1.0000
Epoch 151/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0014 - acc: 1.0000
Epoch 152/250
1500/1500 [==============================] - 0s 25us/step - loss: 0.0014 - acc: 1.0000
Epoch 153/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0013 - acc: 1.0000
Epoch 154/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0013 - acc: 1.0000
Epoch 155/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0012 - acc: 1.0000
Epoch 156/250
1500/1500 [==============================] - 0s 21us/step - loss: 0.0012 - acc: 1.0000
Epoch 157/250
1500/1500 [==============================] - 0s 23us/step - loss: 0.0011 - acc: 1.0000
Epoch 158/250
1500/1500 [==============================] - 0s 22us/step - loss: 0.0011 - acc: 1.0000
Epoc

1500/1500 [==============================] - 0s 23us/step - loss: 1.9707e-04 - acc: 1.0000
Epoch 228/250
1500/1500 [==============================] - 0s 24us/step - loss: 1.9210e-04 - acc: 1.0000
Epoch 229/250
1500/1500 [==============================] - 0s 27us/step - loss: 1.8715e-04 - acc: 1.0000
Epoch 230/250
1500/1500 [==============================] - 0s 25us/step - loss: 1.8406e-04 - acc: 1.0000
Epoch 231/250
1500/1500 [==============================] - 0s 23us/step - loss: 1.7988e-04 - acc: 1.0000
Epoch 232/250
1500/1500 [==============================] - 0s 25us/step - loss: 1.7645e-04 - acc: 1.0000
Epoch 233/250
1500/1500 [==============================] - 0s 23us/step - loss: 1.7218e-04 - acc: 1.0000
Epoch 234/250
1500/1500 [==============================] - 0s 21us/step - loss: 1.6957e-04 - acc: 1.0000
Epoch 235/250
1500/1500 [==============================] - 0s 21us/step - loss: 1.6576e-04 - acc: 1.0000
Epoch 236/250
1500/1500 [==============================] - 0s 21us/st

In [6]:
xs_test.shape

(399, 300)

Evaluating the models for each currency
---

In [7]:
for k in currencies:
    scores = models[k].evaluate(xs_test, classes_test[k])
    print("Currency %s. %s: %.2f%%" % (k, models[k].metrics_names[1], scores[1]*100))    

399/399 [==============================] - 0s 138us/step
Currency BTC. acc: 81.45%
399/399 [==============================] - 0s 141us/step
Currency ETH. acc: 85.96%
399/399 [==============================] - 0s 138us/step
Currency XRP. acc: 70.43%


Testing the predictor
---

Note that 0 is when the candle is < threshold and 1 when candle is >= threshold

In [8]:
print('Prediction', np.round(models[currencies[0]].predict(xs)).astype(int).flatten())
print('Ref:', np.round(classes[currencies[0]], 1))

Prediction [0 0 0 ..., 0 0 0]
Ref: [0 0 0 ..., 0 0 0]


In [9]:
np.round(models[currencies[0]].predict(xs)).astype(int).flatten().sum()

193

In [10]:
np.round(classes[currencies[0]], 1).sum()

181

Testing predictor with new queried data
---

In [12]:
vars = []
for coin in currencies:
    q = get_histo(mhd, coin, block, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)

for k in currencies:
    print('Prediction for currency %s' % k, np.round(models[k].predict(vars.flatten().reshape(1, -1)), 0))

Prediction for currency BTC [[ 0.]]
Prediction for currency ETH [[ 0.]]
Prediction for currency XRP [[ 0.]]
